In [1]:
import os 
import re
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

#Based on example here: https://devklaus.wordpress.com/2018/03/11/sentiment-analysis-on-us-twitter-airlines-dataset-a-deep-learning-approach/

Using TensorFlow backend.


In [2]:
train = pd.read_csv('../data/train.csv',sep=';',index_col=0)
val = pd.read_csv('../data/val.csv',sep=';',index_col=0)
test = pd.read_csv('../data/test.csv',sep=';',index_col=0)

In [3]:
ytrain = pd.get_dummies(train['sentiment']).values
yval = pd.get_dummies(val['sentiment']).values
ytest = pd.get_dummies(test['sentiment']).values
print(Xtrain.shape,ytrain.shape)
print(Xval.shape,yval.shape)
print(Xtest.shape,ytest.shape)

NameError: name 'Xtrain' is not defined

In [13]:
res = []
models = []

In [25]:
max_features = 1000
embed_dim = 128
lstm_out = 100
batch_size = 512
nepochs=7

In [26]:
word_tokenizer = Tokenizer(num_words=max_features, char_level=False)
word_tokenizer.fit_on_texts(train['text'].values.tolist() + val['text'].values.tolist() + test['text'].values.tolist())

X = word_tokenizer.texts_to_sequences(train['text'].values.tolist() + val['text'].values.tolist() + test['text'].values.tolist())
X = pad_sequences(X)
Xtrain = X[:len(train)]
Xval = X[len(train):len(train)+len(val)]
Xtest = X[len(train)+len(val):]

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = Xtrain.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
#print(model.summary())

In [27]:
history = model.fit(Xtrain, ytrain, epochs = nepochs, batch_size=batch_size, validation_data=(Xval, yval))
res += [[nepochs,max_features,embed_dim,lstm_out,batch_size] + history.history['acc'] + history.history['val_acc']]
models += [model]                    

Train on 5000 samples, validate on 1000 samples
Epoch 1/7
5000/5000 [==============================] - 5s 969us/step - loss: 1.0888 - acc: 0.4046 - val_loss: 1.0615 - val_acc: 0.5390
Epoch 2/7
5000/5000 [==============================] - 3s 674us/step - loss: 1.0162 - acc: 0.5550 - val_loss: 0.9871 - val_acc: 0.5320
Epoch 3/7
5000/5000 [==============================] - 3s 663us/step - loss: 0.9217 - acc: 0.5750 - val_loss: 0.9069 - val_acc: 0.6030
Epoch 4/7
5000/5000 [==============================] - 3s 667us/step - loss: 0.7710 - acc: 0.6958 - val_loss: 0.7796 - val_acc: 0.6630
Epoch 5/7
5000/5000 [==============================] - 3s 671us/step - loss: 0.5857 - acc: 0.7784 - val_loss: 0.6754 - val_acc: 0.7150
Epoch 6/7
5000/5000 [==============================] - 3s 679us/step - loss: 0.4504 - acc: 0.8308 - val_loss: 0.6402 - val_acc: 0.7580
Epoch 7/7
5000/5000 [==============================] - 3s 670us/step - loss: 0.3360 - acc: 0.8878 - val_loss: 0.6293 - val_acc: 0.7600


In [33]:
test_pred = []
for pred in model.predict_classes(Xtest):
    temp = [0,0,0]
    temp[pred] = 1
    test_pred += [temp]
    
np.mean(test_pred == ytest)

0.8181818181818182

In [ ]:
plt.plot(range(len(history.history['acc'])),history.history['acc'])
plt.plot(range(len(history.history['val_acc'])),history.history['val_acc'])

In [8]:

nepochs=10
from tqdm import tqdm
with tqdm(total=2*3*4*3) as pbar:
    for max_features in [100,1000]:
        for embed_dim in [16,64,128]:
            for lstm_out in [16,64,128]:
                for batch_size in [32,128,256]:
                    word_tokenizer = Tokenizer(num_words=max_features, char_level=True)
                    word_tokenizer.fit_on_texts(train['text'].values.tolist() + val['text'].values.tolist() + test['text'].values.tolist())


                    X = word_tokenizer.texts_to_sequences(train['text'].values.tolist() + val['text'].values.tolist() + test['text'].values.tolist())
                    X = pad_sequences(X)
                    Xtrain = X[:len(train)]
                    Xval = X[len(train):len(train)+len(val)]
                    Xtest = X[len(train)+len(val):]

                    model = Sequential()
                    model.add(Embedding(max_features, embed_dim,input_length = Xtrain.shape[1]))
                    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
                    model.add(Dense(3,activation='softmax'))
                    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
                    #print(model.summary())

                    history = model.fit(Xtrain, ytrain, epochs = nepochs, batch_size=batch_size, validation_data=(Xval, yval),verbose=False)
                    res += [[nepochs,max_features,embed_dim,lstm_out,batch_size] + history.history['acc'] + history.history['val_acc']]
                    print([nepochs, max_features,embed_dim,lstm_out,batch_size] + [np.max(history.history['acc'])] + [np.max(history.history['val_acc'])])
                    pbar.update()

  1%|▏         | 1/72 [04:07<4:53:13, 247.80s/it]

[10, 100, 16, 16, 32, 0.582, 0.59]


  3%|▎         | 2/72 [05:18<3:47:12, 194.75s/it]

[10, 100, 16, 16, 128, 0.5638, 0.5780000028610229]


  4%|▍         | 3/72 [06:02<2:51:47, 149.38s/it]

...

[10, 1000, 128, 128, 32, 0.6132, 0.604]


 74%|███████▎  | 53/72 [2:52:22<1:25:43, 270.70s/it]

[10, 1000, 128, 128, 128, 0.6102, 0.6040000019073486]


 75%|███████▌  | 54/72 [2:56:58<1:21:40, 272.25s/it]

[10, 1000, 128, 128, 256, 0.6174000001907348, 0.5989999933242798]


In [24]:
res['']

[10000,
 64,
 64,
 128,
 0.4286,
 0.6046,
 0.7566,
 0.8492,
 0.9028,
 0.9308,
 0.9502,
 0.962,
 0.9666,
 0.975,
 0.9798,
 0.9838,
 0.9834,
 0.9824,
 0.986,
 0.987,
 0.9872,
 0.9894,
 0.9876,
 0.9898,
 0.5399999985694885,
 0.6320000014305115,
 0.7100000023841858,
 0.7510000023841857,
 0.7469999990463256,
 0.7419999990463256,
 0.7439999990463256,
 0.7370000014305115,
 0.7240000014305115,
 0.7220000023841858,
 0.7160000023841858,
 0.717000002861023,
 0.709,
 0.7129999990463257,
 0.7089999976158142,
 0.708,
 0.7109999985694885,
 0.7000000023841858,
 0.699,
 0.6980000014305114]

In [9]:
df = pd.DataFrame(np.array(res)[:,:5],columns = ['nepochs','max_features','embed_dim','lstm_out','batch_size'])
tr_acc = pd.DataFrame(np.array(res)[:,5:15])
va_acc = pd.DataFrame(np.array(res)[:,15:25])
df['train_max'] = tr_acc.max(axis=1)
df['val_max'] = va_acc.max(axis=1)
df['diff'] = df['train_max'] - df['val_max']
df[(df['lstm_out']!=3)].sort_values('val_max',ascending=False)

,nepochs,max_features,embed_dim,lstm_out,batch_size,train_max,val_max,diff
48,10.0,1000.0,128.0,64.0,32.0,0.6098,0.610,-0.0002
39,10.0,1000.0,64.0,64.0,32.0,0.6050,0.610,-0.0050
36,10.0,1000.0,64.0,16.0,32.0,0.6010,0.608,-0.0070
43,10.0,1000.0,64.0,128.0,128.0,0.6036,0.608,-0.0044
49,10.0,1000.0,128.0,64.0,128.0,0.6004,0.605,-0.0046
21,10.0,100.0,128.0,64.0,32.0,0.6034,0.605,-0.0016
52,10.0,1000.0,128.0,128.0,128.0,0.6102,0.604,0.0062
51,10.0,1000.0,128.0,128.0,32.0,0.6132,0.604,0.0092
18,10.0,100.0,128.0,16.0,32.0,0.5908,0.604,-0.0132
31,10.0,1000.0,16.0,64.0,128.0,0.5840,0.604,-0.0200


In [10]:
df.to_csv('lstm_char_train_3.csv')

In [42]:
history.history

{'acc': [0.4062000021934509,
  0.5275999977111816,
  0.5857999985694885,
  0.6772000019073486,
  0.7309999984741211,
  0.7759999981880188,
  0.7929999979972839,
  0.8005999985694885,
  0.8136000009536744,
  0.8380000006675721],
 'loss': [1.0746020488739014,
  0.9884465896606446,
  0.8958369854927063,
  0.7637522832870484,
  0.6406505021095276,
  0.5588097978591919,
  0.5105084029197693,
  0.48304083008766174,
  0.46273315505981444,
  0.43337293162345886],
 'val_acc': [0.5220000109672547,
  0.5660000066757203,
  0.6179999914169312,
  0.672000009059906,
  0.7129999985694885,
  0.7319999976158142,
  0.7400000061988831,
  0.7509999952316284,
  0.7349999890327453,
  0.7570000038146972],
 'val_loss': [1.0360965976715089,
  0.9714739093780518,
  0.8775711927413941,
  0.7391939158439637,
  0.678727029800415,
  0.6424490041732788,
  0.6495523729324341,
  0.6502507195472718,
  0.6688855495452881,
  0.7119390687942505]}